# Fast simulation perfromance parameterization 
Expert  configurable event display for fast simulzation studies  


## Contents 
[Import modules](#import) <br>
[Load trees, set aliases](#load) <br>
[XYZ display pt](#XYZdisplayPt) <br>
[XYZ display pt](#XYZdisplaytrack) <br>

In [ ]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter, RandomForest
import plotly.express as px
import time
import pickle
from IPython.display import Image, clear_output
from RootInteractive.InteractiveDrawing.bokeh.CDSCompress import *
#
from fastSimulation import *

## 1.) Load trees and define derived variables:
<a id='load'></a>

* Covaraince matrix of Alice kalman track - stored as an array 
* ```(y,z,sing*phi),tgl(theta),qPt) -> (0,2,5,9,14)```
 * 0   
 * 1 2
 * 3 4 5 
 * 6 7 8 9
 * 10 11 12 13 14 

In [ ]:
#inputData = "/home2/miranov/github/fastMCKalman/data/alice3/fastParticle.list";
#inputData = "${fastMCKalman}/data/alice3/fastParticle.list";
inputData = "/home2/miranov/github/fastMCKalman/data/testTPCSA/fastParticle.list"
#inputData = "/home2/miranov/github/fastMCKalman/data/run2/fastParticle.list";

In [ ]:
ROOT.initTreeFast(inputData)
tree=ROOT.treeFast

In [ ]:
tree.SetMarkerStyle(25)
tree.SetMarkerSize(0.5)
ROOT.setAliases(tree)
tree.SetAlias("qPt","part.fParamMC[0].fP[4]")
tree.SetAlias("ptMC","part.fParamMC[0].fData.Pt()")
tree.SetAlias("ptIn","part.fParamIn[1].fData.Pt()")
tree.SetAlias("ptMCP","part.fParamMC[].fData.Pt()")
tree.SetAlias("tglMC","part.fParamMC[0].fP[3]")
tree.SetAlias("tglIn","part.fParamIn[1].fP[3]")
#
tree.SetAlias("sigmaY0","sqrt(part.fParamIn[1].fC[0])")
tree.SetAlias("sigmaZ0","sqrt(part.fParamIn[1].fC[2])")
tree.SetAlias("sigmaqPt0","sqrt(part.fParamIn[1].fC[14])")
#
tree.SetAlias("sigmaY0Rot","sqrt(part.fParamInRot[1].fC[0])")
tree.SetAlias("sigmaZ0Rot","sqrt(part.fParamInRot[1].fC[2])")
tree.SetAlias("sigmaqPt0Rot","sqrt(part.fParamInRot[1].fC[14])")
#
tree.SetAlias("gzMC","(part.fParamMC[].fP[1])")
tree.SetAlias("gzIn","(part.fParamIn[].fP[1])")
tree.SetAlias("rMC","sqrt(gxMC**2+gyMC**2)")
tree.SetAlias("snpMC","(part.fParamMC[].fP[2])")
tree.SetAlias("pIndex","Iteration$")
tree.SetAlias("trackID","i")
tree.SetAlias("snpMax","abs(part.fParamMC[fMaxLayer].fP[2])")
#
tree.SetAlias("meanptMC","part.getMean(0+0,0)");
tree.SetAlias("meanptMCInv","part.getMean(1+0,0)");
tree.SetAlias("meandEdxExp","part.getMean(2+0,0)");
tree.SetAlias("meandEdxExpInv","part.getMean(3+0,0)");
tree.SetAlias("X0Norm","geom.fLayerX0.fData[0]/7.8350968e-05");
tree.SetAlias("sigmaRPhi","geom.fLayerResolRPhi.fData[0]");
tree.SetAlias("sigmaPtRel0","sigmaqPt0*ptMC");
tree.SetAlias("sigmaPtRel0Mean","sigmaqPt0/meanptMCInv");
#
tree.SetAlias("fitCut","ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5")


In [ ]:
c=ROOT.TCanvas()

In [ ]:
%%time
tree.SetAlias("fitCut","ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5&&fMaxLayer>2")
defaultCutTrack="fitCut"
kineParam=["sigmaPtRel0","qPt","ptMC",
                           "sigmaY0","sigmaZ0","sigmaqPt0",
                           "sigmaY0Rot","sigmaZ0Rot","sigmaqPt0Rot",
                           "pidCode","isSecondary","fMaxLayer","dEdxExp",
                          "snpMax","fMassMC"]
resolParam=["meanptMCInv","meandEdxExpInv","X0Norm","tglMC","Larm","sigmaRPhi"]
dfRes=tree2Panda(tree,kineParam+resolParam,
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=10000)


In [ ]:
dfRes.shape

In [ ]:
#dfRes[variableX].head(5)

In [ ]:
%%time
dfResML=dfRes
npoints=dfRes.shape[0]
varFit='sigmaPtRel0'
variableX=resolParam
dataContainer = DataContainer(dfResML, variableX, varFit, [npoints//2,npoints//2])
fitter = Fitter(dataContainer)
fitter.Register_Method('RF','RandomForest', 'Regressor', n_estimators=200, max_depth=12)
fitter.Register_Method('RF_10','RandomForest', 'Regressor', n_estimators=200, max_depth=10)

fitter.Fit()
fitter.printImportance()
#
for method in ['RF' , 'RF_10']:    
    dfResML= fitter.AppendOtherPandas(method,dfResML)
fitter.AppendStatPandas("RF",dfResML)
dfResML=fitter.AppendStatPandas("RF_10",dfResML)


In [ ]:
dfResMLDraw=dfResML
dfResMLDraw["FitRatio"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")

In [ ]:
dfResML.columns

## Resolution plot - outliers in covariance matrix could be too small
* resolution at layer 0 as obtained by covariance matrix 
* problem similar in 2 rotation schema
  * rotate to local after each propagation
  * only one rotation
  
###resolParam=["meanptMCInv","meandEdxExpInv","X0Norm","tglMC","Larm","sigmaRPhi"]  

In [ ]:
arrayCompressionRelative10=[(".*",[("relative",10), ("code",0), ("zip",0), ("base64",0)])]
arrayCompressionRelative8=[(".*",[("relative",8), ("code",0), ("zip",0), ("base64",0)])]
output_file("fastSimulDisplayResolution.html")
dfResMLDraw=dfResML
#
dfResMLDraw["FitRatio"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")
dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*0.01*(sigmaRPhi+0.0001)*sqrt(720./(Larm+4))/(0.3*0.5*(Larm/100.)**(2.)))")
dfResMLDraw["ptResolHPT0"]=dfResMLDraw.eval("(sigmaRPhi+0.0001)/((Larm/100.)**2.5)")
dfResMLDraw["pMC"]=dfResMLDraw.eval("sqrt(1+tglMC**2)*ptMC")
dfResMLDraw["Beta"]=dfResMLDraw.eval("sqrt(pMC**2/(pMC**2+fMassMC**2))")
dfResMLDraw["ptResolLPT"]=dfResMLDraw.eval("(0.2*0.5/Beta)*sqrt(Larm/100.)")
dfResMLDraw["FitRatioHPT"]=dfResMLDraw.eval("log(sigmaPtRel0/ptResolHPT)")
dfResMLDraw["FitRatioLPT"]=dfResMLDraw.eval("log(sigmaPtRel0/ptResolLPT)")
# define categorical
dfResMLDraw["isHighPt"]=dfResMLDraw.eval("ptMC>2")
dfResMLDraw["X0NormC"]=dfResMLDraw["X0Norm"].astype("int").astype("category")
dfResMLDraw["sigmaRPhiC"]=dfResMLDraw.eval("sigmaRPhi/0.025").astype("int").astype("category")

In [ ]:
parameterArray = [
    {"name": "colorZPt", "value":"Larm", "options":["dEdxExp","X0Norm","pidCode","Larm","ptResolHPT","ptResolLPT","sigmaRPhi","ptResolHPT0"]},
    {"name": "colorZqPt", "value":"Larm", "options":["dEdxExp","X0Norm","pidCode","Larm","ptResolHPT","ptResolLPT","sigmaRPhi"]},
    {"name": "markerSize", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px", "17px", "19px"]},  
]

widgetParams=[
    #
    ['select',["colorZPt"], {"callback": "parameter", "default": 2}],
    ['slider',["markerSize"], {"callback": "parameter"}],
    ['select',["legendFontSize"], {"callback": "parameter", "default": 2}],
    #infironment
    ['range', ['dEdxExp']],
    ['range', ['meandEdxExpInv']],
    ['range', ['X0Norm']],
    ['range', ['pidCode']],
    ['range', ['sigmaRPhi']],
    #
    ['range', ['ptMC']],
    ['range', ['qPt']],
    ['range', ['tglMC']],
    ['range', ['Larm']],
    #
    ["range", ['FitRatio']],
    ["range", ['FitRatioHPT']],
    ["range", ['ptResolHPT']],
    #
    ['multiSelect',["isHighPt"]],
    ['multiSelect',["X0NormC"]],
    ['multiSelect',["sigmaRPhiC"]],
    ['multiSelect',["pidCode"]],
#    ['multiSelect',["sigmaRPhiC"]],
    #dummy
]
widgetLayoutDescT0=[
                    [3,4,5,6,7],
                    [8,9,10,11],
                    [12,13,14],
                     [15,16,17,18],
                    {'sizing_mode':'scale_width',"legend_visible":True} ]
widgetLayoutDesc={
    "Selection": widgetLayoutDescT0,
    "Graphics": [[0,1,2], {'sizing_mode': 'scale_width'}]
}

In [ ]:
histoArray = [
    #
    {"name": "histoqPtLarm", "variables": ["sigmaPtRel0","Larm"], "nbins": [50,20]},
    {"name": "histoqPtSRPhi", "variables": ["sigmaPtRel0","sigmaRPhi"], "nbins": [50,20]},
    #
    {"name": "histoqPt", "variables": ["qPt"], "nbins": 50},
    {"name": "histoFitRatio", "variables": ["FitRatio"], "nbins": 50},
    {"name": "histoFitRatioHPT", "variables": ["FitRatioHPT"], "nbins": 50},
    {"name": "histoFitRatioLPT", "variables": ["FitRatioLPT"], "nbins": 50},
    #

]

figureArray = [
    [['qPt'], ['sigmaPtRel0'], {"colorZvar":"colorZPt"}],
    [['qPt'], ['RF'], {"colorZvar":"colorZPt"}],
    [['qPt'], ['ptResolHPT'], {"colorZvar":"colorZPt"}],
    [['qPt'], ['ptResolLPT'], {"colorZvar":"colorZPt"}],
    #
    [['ptMC'], ['sigmaPtRel0'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['RF'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['ptResolHPT'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['ptResolLPT'], {"colorZvar":"colorZPt"}],
    #
    [['qPt'], ['FitRatio'], {"colorZvar":"pidCode"}],
    [['qPt'], ['FitRatioHPT'], {"colorZvar":"pidCode"}],
    [['qPt'], ['FitRatioLPT'], {"colorZvar":"pidCode"}],
    #
    [["qPt"],["histoqPt"],{}],
    [["FitRatio"],["histoFitRatio"],{}],
    [["FitRatioHPT"],["histoFitRatioHPT"],{}],
    [["FitRatioLPT"],["histoFitRatioLPT"],{}],
    ["tableHisto", {"rowwise": False}],
    ['table'],
    {"size": 5}
]

tooltips = [("qPt","@qPt"),("sigmaRPhi","@sigmaRPhi")]
figureLayoutDescT0=[ [0,1,  {'plot_height':200, "commonX":3}],     [2,3,  {'plot_height':200, "commonX":3}],       {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}]
figureLayoutDescT1=[ [4,5,  {'plot_height':200, "commonX":7}],     [6,7,  {'plot_height':200, "commonX":7}],       {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}]
figureLayoutDescT2=[
    [8,9,  {'plot_height':200}],    
    [10,11,  {'plot_height':200}],       
    {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}
]
figureLayoutDesc={
    "pt Resol tab": figureLayoutDescT0,
    "pt Resol ratio tab": figureLayoutDescT1,
    "Histos": figureLayoutDescT2, 
}
output_file("fastSimulDisplayResolution.html")
figC=bokehDrawSA.fromArray(dfResMLDraw, "fMaxLayer>=0&ptMC<50", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=2000,
                           rescaleColorMapper=True,arrayCompression=arrayCompressionRelative8,histogramArray=histoArray,parameterArray=parameterArray)

In [ ]:
dfResMLDraw["pMC"]=dfResMLDraw.eval("sqrt(1+tglMC**2)*ptMC")
dfResMLDraw["Beta"]=dfResMLDraw.eval("sqrt(pMC**2/(pMC**2+fMassMC**2))")
dfResMLDraw["ptResolLPT"]=dfResMLDraw.eval("(0.2*0.5/Beta)*sqrt(Larm/100.)")

In [ ]:
dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*(sigmaRPhi+0.01)*sqrt(720./(Larm+4.))/(0.3*0.5*(Larm/100.)**2.))")

# Fast prototyping part

In [ ]:

dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*sigmaRPhi*sqrt(720.)/(0.5*Larm**(2.5)))")

In [ ]:
dfResMLDraw["X0Norm"].hist()

In [ ]:
dfResMLDraw

In [ ]:
# Resolut

In [ ]:
# t1. Resolution scaling does not work at all for secondaries
# t2. Resolution scaling works like expected (L**2.5) down to Larm 100 cm, below it is breaking 
#t1.
#tree.Draw("sigmaqPt0*(sigmaRPhi+0.001)*Larm**2.5:Larm>>hisLArm(25,0,250)","fitCut&&Larm>5&&ptMC>10&&(isSecondary)","prof")
#t2.
tree.Draw("sigmaqPt0*(sigmaRPhi+0.001)*Larm**2.5:Larm>>hisLArm(25,0,250)","fitCut&&Larm>5&&ptMC>10&&(!isSecondary)","prof")
c.Draw("")

In [ ]:
tree.GetAlias("Larm")